# GitHub Graph and README.md Similarity EDA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
from collections import deque
from itertools import product
plt.style.use('ggplot')
%matplotlib inline

In [1]:
# Standard Python Library for working with graphs
import networkx as nx

# Helper library to visualize graphs
import nxpd as nxpd

# Helper library for community related functions
import community



ImportError: No module named nxpd

In [ ]:
df = pd.read_csv('data/test_data/followers10k.csv')
df.head()

In [ ]:
df.info()

In [ ]:
#Just grab ten edges
graph_small = nx.from_pandas_dataframe(df.iloc[:10,:], source='follower_id', target='user_id', create_using=None)

In [ ]:
nxpd.draw(graph_small, show='ipynb')

In [ ]:
#Grab 100 edges
graph100 = nx.from_pandas_dataframe(df.iloc[:100,:], source='follower_id', target='user_id', 
                                    create_using=nx.DiGraph())


In [ ]:
nx.write_gexf(graph100, 'data/test_data/followers100graph.gexf')

## Use py2neo4j to traverse the github graph

In [18]:
from py2neo import Graph

In [19]:
graph = Graph("http://localhost:7474/db/data/", password="0")

In [27]:
# add_constraint_repo = '''
# CREATE CONSTRAINT ON (o:Repo) ASSERT o.repoID IS UNIQUE;
# '''
# add_constraint_user = '''
# CREATE CONSTRAINT ON (p:User) ASSERT p.userID IS UNIQUE;
# '''
# graph.run(add_constraint_repo)
# graph.run(add_constraint_user)

In [31]:
# add_repo_nodes = '''
# USING PERIODIC COMMIT
# LOAD CSV WITH HEADERS FROM
# "file:///home/ubuntu/db/data/active_projects.csv" AS row
# FIELDTERMINATOR '\t'
# CREATE (:Repo {repoID: toInt(row.id),
#                url: row.url,
#                userID: toInt(row.owner_id),
#                name: row.name,
#                description: row.description,
#                mainLanguage: row.language,
#                repoCreated: row.created_at,
#                forkedFrom: toInt(row.forked_from)
#                });
# '''
# graph.run(add_repo_nodes)

In [32]:
# add_user_nodes = '''
# USING PERIODIC COMMIT
# LOAD CSV WITH HEADERS
# FROM "file:///home/ubuntu/db/data/active_users.csv" AS row
# FIELDTERMINATOR '\t'
# CREATE (:User {userID: toInt(row.id),
#                login: row.login,
#                company: row.company,
#                userCreated: row.created_at,
#                type: row.type,
#                countryCode: row.country_code,
#                state: row.state,
#                city: row.city
#                });
# '''
# graph.run(add_user_nodes)

In [35]:
# create_indices = [
# 'CREATE INDEX ON :User(login);',
# 'CREATE INDEX ON :Repo(forkedFrom);']

# for index in create_indices:
#     graph.run(index)

In [36]:
# relationships = [
#     '''
#     // Create relationship between follower to user (user-user)
#     USING PERIODIC COMMIT
#     LOAD CSV WITH HEADERS FROM "file:///home/ubuntu/db/data/followers.csv" AS row
#     MATCH (follower:User {userID: toInt(row.follower_id)})
#     MATCH (user:User {userID: toInt(row.user_id)})
#     MERGE (follower)-[:FOLLOWS]->(user);''',
#     '''
#     // Create relationship between original repo and forks (repo-repo)
#     USING PERIODIC COMMIT
#     LOAD CSV WITH HEADERS FROM "file:///home/ubuntu/db/data/active_projects.csv" AS row
#     FIELDTERMINATOR '\t'
#     MATCH (fork:Repo {repoID: toInt(row.id)})
#     MATCH (original:Repo {repoID: toInt(row.forked_from)})
#     MERGE (fork)-[:FORKED_FROM]->(original);''',
#     '''
#     // Create relationship between user and owned repos (user-repo)
#     USING PERIODIC COMMIT
#     LOAD CSV WITH HEADERS FROM "file:///home/ubuntu/db/data/active_projects.csv" AS row
#     FIELDTERMINATOR '\t'
#     MATCH (user:User {userID: toInt(row.owner_id)})
#     MATCH (repo:Repo {repoID: toInt(row.id)})
#     MERGE (user)-[:OWNS]->(repo);''',
#     '''
#     // Create relationship between user and org user (user-user)
#     USING PERIODIC COMMIT
#     LOAD CSV WITH HEADERS FROM "file:///home/ubuntu/db/data/organization_members.csv" AS row
#     MATCH (user:User {userID: toInt(row.user_id)})
#     MATCH (org:User {userID: toInt(row.org_id)})
#     MERGE (user)-[:MEMBER_OF]->(org);''',
#     '''
#     // Create relationship between user and repo stars (user-repo)
#     USING PERIODIC COMMIT
#     LOAD CSV WITH HEADERS FROM "file:///home/ubuntu/db/data/watchers.csv" AS row
#     MATCH (user:User {userID: toInt(row.user_id)})
#     MATCH (repo:Repo {repoID: toInt(row.repo_id)})
#     MERGE (user)-[:STARRED]->(repo);'''
# ]

# for relation in relationships:
#     graph.run(relation)

In [20]:
# Return a user and its immediate relationships
query = """
MATCH (a:User {login: {username}})-[r]-(b)
RETURN a, r, b
"""
data = graph.run(query, username="candlepin")

for d in data:
    print d['b']
    print(" ")

(f82bc02:Repo {description:"The Candlepin entitlement engine",mainLanguage:"Java",name:"candlepin",repoCreated:"2012-06-07 14:58:35",repoID:2396,url:"https://api.github.com/repos/candlepin/candlepin",userID:9548})
 
(b37fddd:Repo {description:"A GUI and CLI client for Candlepin",mainLanguage:"Python",name:"subscription-manager",repoCreated:"2012-05-17 16:13:30",repoID:1808,url:"https://api.github.com/repos/candlepin/subscription-manager",userID:9548})
 
(aaafe0d:User {city:"Halifax",company:"Red Hat",countryCode:"ca",login:"dgoodwin",state:"NS",type:"USR",userCreated:"2009-02-02 23:05:30",userID:11443})
 
(a3b8ebc:User {city:"\\N",company:"\\N",countryCode:"\\N",login:"jmrodri",state:"\\N",type:"USR",userCreated:"2008-07-15 16:55:30",userID:10612})
 
(c31790b:User {city:"Raleigh",company:"Red Hat",countryCode:"us",login:"awood",state:"NC",type:"USR",userCreated:"2008-07-22 18:44:25",userID:9551})
 
(e8fe88c:User {city:"\\N",company:"\\N",countryCode:"\\N",login:"alikins",state:"\\N",ty

In [21]:
# Returns starred repos
query = '''
MATCH (:User {login: {username}})-[:STARRED]->(repo:Repo)
RETURN repo;
'''
data = graph.run(query, username="candlepin")

for d in data:
    print d
    print(" ")

(u'repo': (f82bc02:Repo {description:"The Candlepin entitlement engine",mainLanguage:"Java",name:"candlepin",repoCreated:"2012-06-07 14:58:35",repoID:2396,url:"https://api.github.com/repos/candlepin/candlepin",userID:9548}))
 
(u'repo': (b37fddd:Repo {description:"A GUI and CLI client for Candlepin",mainLanguage:"Python",name:"subscription-manager",repoCreated:"2012-05-17 16:13:30",repoID:1808,url:"https://api.github.com/repos/candlepin/subscription-manager",userID:9548}))
 


In [24]:
# Returns forked repos
query = '''
MATCH (:User {login: {username}})-[:FORKED_FROM]->(forked:Repo)
MATCH (:User {login: {username}})-[:STARRED]->(starred:Repo)
RETURN forked, starred ;
'''
data = graph.run(query, username="candlepin")

for d in data:
    print d
    print(" ")

## Read in all readme files

In [2]:
import pandas as pd
import numpy as np

In [117]:
readme1 = pd.read_csv("/home/ubuntu/db/github-collaborator/data/readmes/result-Tue-09-05-1250AM.csv", header=None)
readme1 = readme1.rename(columns={0:"login", 1:"repo", 2:"readme_content"})
readme1.head(3)

,login,repo,readme_content
0,Electronics-Ninja,BBB_python,# BBB_python\n\n# Testing\n
1,d0u9,.dot,# .dot\nThe bucket of dot files.\n\n# Descript...
2,candlepin,subscription-manager,subscription-manager\n====================\n\n...


In [118]:
readme2 = pd.read_csv("/home/ubuntu/db/github-collaborator/data/readmes/result-Tue-09-05-1252AM.csv", header=None)
readme2 = readme2.rename(columns={0:"login", 1:"repo", 2:"readme_content"})
readme2.head(3)

,login,repo,readme_content
0,nsc-norway,lims,# lims\nMiscellaneous scripts for Clarity LIMS...
1,nsc-norway,pipeline,# pipeline\n\nScripts for initial processing o...
2,Stufinite,gluttony,# GluttonyTw（暴食）\n\n今天吃什麼的api\n\n_`gluttony`_ ...


In [119]:
readme3 = pd.read_csv("/home/ubuntu/db/github-collaborator/data/readmes/result-Tue-09-05-1255AM.csv", header=None)
readme3 = readme3.rename(columns={0:"login", 1:"repo", 2:"readme_content"})
readme3.head(3)

,login,repo,readme_content
0,arun24816,dcos-cassandra-service-1,# DC/OS Apache Cassandra Service Documentation...
1,LipuFei,cura-god-mode-plugin,# cura-god-mode-plugin\nPlugin for Cura to mak...
2,mbartsch,docker-graphite-statsd,[![Docker Pulls](https://img.shields.io/docker...


In [120]:
readme4 = pd.read_csv("/home/ubuntu/db/github-collaborator/data/readmes/result-Tue-09-05-1258AM.csv", header=None)
readme4 = readme4.rename(columns={0:"login", 1:"repo", 2:"readme_content"})
readme4.head(3)

,login,repo,readme_content
0,amanbh,indivisible,# indivisible\nAggregating call to action site...
1,xgfn,gdb-dashboard,GDB dashboard\n=============\n\nModular visual...
2,parmarnikunjkumar,async-http-client,Async Http Client ([@AsyncHttpClient](https://...


In [121]:
frames = [readme1, readme2, readme3, readme4]

In [122]:
readmes = pd.concat(frames)
readmes.head(10)

,login,repo,readme_content
0,Electronics-Ninja,BBB_python,# BBB_python\n\n# Testing\n
1,d0u9,.dot,# .dot\nThe bucket of dot files.\n\n# Descript...
2,candlepin,subscription-manager,subscription-manager\n====================\n\n...
3,greenalpha,utxx,# utxx - an open-source collection of C++ util...
4,endlessm,linux,NaN
5,khssb,core-project,NaN
6,aperaid,xana,# Laravel PHP Framework\n\n[![Build Status](ht...
7,basti-shi031,ApplicationRelease,# ReadMe\n* python 3.5\n* win 8\n\n## How to u...
8,harishkrupo,device-androidia-mixins,NaN
9,agabardo,gitfiti,NaN


In [9]:
#Missing 691 projects. Should have a total of 228,180
readmes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227489 entries, 0 to 56489
Data columns (total 3 columns):
login             227489 non-null object
repo              227483 non-null object
readme_content    148614 non-null object
dtypes: object(3)
memory usage: 6.9+ MB


In [11]:
users_repos = pd.read_csv('../data/users_projects.csv')
users_repos.tail()

,login,name
227484,cbott,RecipeFinder
227485,borfast,housing-reviews
227486,ledettwy,fabtests
227487,Harhro94,disentanglement
227488,jherico,hifi_tests


## Join user_id and repo_id to readmes df

In [123]:
readmes['user_repo'] = readmes['login'] + '/' + readmes['repo']
readmes.head()

,login,repo,readme_content,user_repo
0,Electronics-Ninja,BBB_python,# BBB_python\n\n# Testing\n,Electronics-Ninja/BBB_python
1,d0u9,.dot,# .dot\nThe bucket of dot files.\n\n# Descript...,d0u9/.dot
2,candlepin,subscription-manager,subscription-manager\n====================\n\n...,candlepin/subscription-manager
3,greenalpha,utxx,# utxx - an open-source collection of C++ util...,greenalpha/utxx
4,endlessm,linux,NaN,endlessm/linux


In [124]:
repos = pd.read_csv('../data/active_projects.csv', sep='\t')
repos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228180 entries, 0 to 228179
Data columns (total 7 columns):
id             228180 non-null int64
url            228180 non-null object
owner_id       228180 non-null int64
name           228174 non-null object
language       228180 non-null object
created_at     228180 non-null object
forked_from    228180 non-null object
dtypes: int64(2), object(5)
memory usage: 12.2+ MB


In [125]:
#There are 6 repos that are named "NaN" or "NULL" so don't join. Replaced with their names
repos[repos['name'].isnull()]

,id,url,owner_id,name,language,created_at,forked_from
24202,20076735,https://api.github.com/repos/nodejs/nan,667293,NaN,C++,2013-07-20 11:12:03,\N
35940,59605172,https://api.github.com/repos/mkrufky/nan,307037,NaN,C++,2017-03-13 22:37:23,20076735
42615,7256895,https://api.github.com/repos/bnoordhuis/nan,963,NaN,Python,2014-01-07 15:47:47,20076735
68682,34585350,https://api.github.com/repos/ilib0x00000000/NULL,8376310,NaN,C,2016-04-08 07:32:05,\N
174186,66121500,https://api.github.com/repos/AndyleeSharp/nan,5069255,NaN,C++,2017-06-01 05:59:57,20076735
182499,66471604,https://api.github.com/repos/Wulf/nan,11554306,NaN,C++,2017-06-05 21:03:21,20076735


In [126]:
#repos.iloc[68682,3] = 'NULL'
repos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228180 entries, 0 to 228179
Data columns (total 7 columns):
id             228180 non-null int64
url            228180 non-null object
owner_id       228180 non-null int64
name           228174 non-null object
language       228180 non-null object
created_at     228180 non-null object
forked_from    228180 non-null object
dtypes: int64(2), object(5)
memory usage: 12.2+ MB


In [127]:
users = pd.read_csv('../data/active_users.csv', sep='\t')
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159321 entries, 0 to 159320
Data columns (total 9 columns):
id              159321 non-null int64
login           159321 non-null object
company         159289 non-null object
created_at      159321 non-null object
type            159321 non-null object
country_code    159321 non-null object
state           159321 non-null object
city            159321 non-null object
location        159319 non-null object
dtypes: int64(1), object(8)
memory usage: 10.9+ MB


In [128]:
joined = pd.merge(repos, users, how='left', left_on='owner_id', right_on='id', 
                  suffixes=('_repo', '_user'))
joined = joined.drop(['url', 'owner_id', 'language', 'created_at_repo', 
             'forked_from', 'company', 'created_at_user', 'type', 'country_code', 
             'state', 'city', 'location'], axis=1)

In [129]:
joined['user_repo'] = joined['login'] + '/' + joined['name']
joined.head()

,id_repo,name,id_user,login,user_repo
0,27750972,BBB_python,7259404.0,Electronics-Ninja,Electronics-Ninja/BBB_python
1,28683824,.dot,8629487.0,d0u9,d0u9/.dot
2,1808,subscription-manager,9548.0,candlepin,candlepin/subscription-manager
3,28922584,utxx,10557672.0,greenalpha,greenalpha/utxx
4,9884164,linux,2234193.0,endlessm,endlessm/linux


In [130]:
# Now I'm missing login names of users?
#Missing 691 user logins
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228180 entries, 0 to 228179
Data columns (total 5 columns):
id_repo      228180 non-null int64
name         228174 non-null object
id_user      227489 non-null float64
login        227489 non-null object
user_repo    227483 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 10.4+ MB


In [131]:
joined[joined['user_repo'].isnull()]

,id_repo,name,id_user,login,user_repo
587,60627079,eyetracking,NaN,NaN,NaN
684,8950098,volttron,NaN,NaN,NaN
1319,56441120,tock,NaN,NaN,NaN
4970,56878754,competitive_programming,NaN,NaN,NaN
5607,60593568,mxzero,NaN,NaN,NaN
6209,60509056,edx6.001x,NaN,NaN,NaN
6898,60970517,android_vendor_cm,NaN,NaN,NaN
7043,55954313,HomeAssistant,NaN,NaN,NaN
7100,59695604,famestats_redesigned,NaN,NaN,NaN
8015,55643841,JwlSelAnd,NaN,NaN,NaN


In [114]:
joined[joined['name'] == 'HomeAssistant']

,id_repo,name,id_user,login,user_repo
7043,55954313,HomeAssistant,NaN,NaN,NaN


In [132]:
#Just going to drop the repos without a user login, since I won't be able to join the README content
joined = joined.dropna(axis=0, subset=['user_repo'])
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227483 entries, 0 to 228179
Data columns (total 5 columns):
id_repo      227483 non-null int64
name         227483 non-null object
id_user      227483 non-null float64
login        227483 non-null object
user_repo    227483 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 10.4+ MB


In [133]:
readme_content = pd.merge(joined, readmes, how='left', left_on='user_repo', right_on='user_repo', 
                  suffixes=('_joined', '_readme'))
#readmes = readmes.drop(['name', 'login_joined'], axis=1)
readme_content.head(5)

,id_repo,name,id_user,login_joined,user_repo,login_readme,repo,readme_content
0,27750972,BBB_python,7259404.0,Electronics-Ninja,Electronics-Ninja/BBB_python,Electronics-Ninja,BBB_python,# BBB_python\n\n# Testing\n
1,28683824,.dot,8629487.0,d0u9,d0u9/.dot,d0u9,.dot,# .dot\nThe bucket of dot files.\n\n# Descript...
2,1808,subscription-manager,9548.0,candlepin,candlepin/subscription-manager,candlepin,subscription-manager,subscription-manager\n====================\n\n...
3,28922584,utxx,10557672.0,greenalpha,greenalpha/utxx,greenalpha,utxx,# utxx - an open-source collection of C++ util...
4,9884164,linux,2234193.0,endlessm,endlessm/linux,endlessm,linux,NaN


In [134]:
readme_content.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227483 entries, 0 to 227482
Data columns (total 8 columns):
id_repo           227483 non-null int64
name              227483 non-null object
id_user           227483 non-null float64
login_joined      227483 non-null object
user_repo         227483 non-null object
login_readme      227483 non-null object
repo              227483 non-null object
readme_content    148608 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 15.6+ MB


In [135]:
readme_content.columns

Index([u'id_repo', u'name', u'id_user', u'login_joined', u'user_repo',
       u'login_readme', u'repo', u'readme_content'],
      dtype='object')

In [136]:
pwd

u'/home/ubuntu/db/github-collaborator'

In [137]:
readme_content.to_pickle('/home/ubuntu/db/github-collaborator/data/readme_df.pkl')

## Replace basic markdown and underscores to free up words

In [138]:
readmes = pd.read_pickle('/home/ubuntu/db/github-collaborator/data/readme_df.pkl')
readmes.head(2)

,id_repo,name,id_user,login_joined,user_repo,login_readme,repo,readme_content
0,27750972,BBB_python,7259404.0,Electronics-Ninja,Electronics-Ninja/BBB_python,Electronics-Ninja,BBB_python,# BBB_python\n\n# Testing\n
1,28683824,.dot,8629487.0,d0u9,d0u9/.dot,d0u9,.dot,# .dot\nThe bucket of dot files.\n\n# Descript...


In [139]:
readmes.readme_content = readmes['readme_content'].replace(to_replace='[-_#=><0-9]', value=' ', regex=True) 
readmes.head(5)

,id_repo,name,id_user,login_joined,user_repo,login_readme,repo,readme_content
0,27750972,BBB_python,7259404.0,Electronics-Ninja,Electronics-Ninja/BBB_python,Electronics-Ninja,BBB_python,BBB python\n\n Testing\n
1,28683824,.dot,8629487.0,d0u9,d0u9/.dot,d0u9,.dot,.dot\nThe bucket of dot files.\n\n Descript...
2,1808,subscription-manager,9548.0,candlepin,candlepin/subscription-manager,candlepin,subscription-manager,subscription manager\n \n\n...
3,28922584,utxx,10557672.0,greenalpha,greenalpha/utxx,greenalpha,utxx,utxx an open source collection of C++ util...
4,9884164,linux,2234193.0,endlessm,endlessm/linux,endlessm,linux,NaN


In [140]:
readmes.readme_content = readmes['readme_content'].replace(to_replace='(\\r|\\n|\\r\\n)+', value=' ', regex=True) 
readmes.head(5)

,id_repo,name,id_user,login_joined,user_repo,login_readme,repo,readme_content
0,27750972,BBB_python,7259404.0,Electronics-Ninja,Electronics-Ninja/BBB_python,Electronics-Ninja,BBB_python,BBB python Testing
1,28683824,.dot,8629487.0,d0u9,d0u9/.dot,d0u9,.dot,.dot The bucket of dot files. Description ...
2,1808,subscription-manager,9548.0,candlepin,candlepin/subscription-manager,candlepin,subscription-manager,subscription manager The ...
3,28922584,utxx,10557672.0,greenalpha,greenalpha/utxx,greenalpha,utxx,utxx an open source collection of C++ util...
4,9884164,linux,2234193.0,endlessm,endlessm/linux,endlessm,linux,NaN


## Decode unicode so we can feed the content into the Vectorizer later

In [142]:
readmes['readme_content'] =  readmes['readme_content'].str.replace('[^\x00-\x7F]',' ')
readmes.head(3)

,id_repo,name,id_user,login_joined,user_repo,login_readme,repo,readme_content
0,27750972,BBB_python,7259404.0,Electronics-Ninja,Electronics-Ninja/BBB_python,Electronics-Ninja,BBB_python,BBB python Testing
1,28683824,.dot,8629487.0,d0u9,d0u9/.dot,d0u9,.dot,.dot The bucket of dot files. Description ...
2,1808,subscription-manager,9548.0,candlepin,candlepin/subscription-manager,candlepin,subscription-manager,subscription manager The ...


## Replace any missing README content with the mainLanguage used in the repo

In [144]:
repos = pd.read_csv('../data/active_projects.csv', sep='\t')
repos.head()

,id,url,owner_id,name,language,created_at,forked_from
0,27750972,https://api.github.com/repos/Electronics-Ninja...,7259404,BBB_python,Python,2015-10-22 01:13:11,\N
1,28683824,https://api.github.com/repos/d0u9/.dot,8629487,.dot,VimL,2015-11-29 10:34:32,\N
2,1808,https://api.github.com/repos/candlepin/subscri...,9548,subscription-manager,Python,2012-05-17 16:13:30,\N
3,28922584,https://api.github.com/repos/greenalpha/utxx,10557672,utxx,C++,2015-11-17 16:14:36,2149824
4,9884164,https://api.github.com/repos/endlessm/linux,2234193,linux,C,2014-05-20 18:43:21,\N


In [145]:
readmes_j = pd.merge(readmes, repos, how='left', left_on='user_repo', right_on='user_repo', 
                  suffixes=('_joined', '_readme'))

,id_repo,name,id_user,login_joined,user_repo,login_readme,repo,readme_content,name_repo,language
0,27750972,BBB_python,7259404.0,Electronics-Ninja,Electronics-Ninja/BBB_python,Electronics-Ninja,BBB_python,BBB python Testing,BBB_python,Python
1,28683824,.dot,8629487.0,d0u9,d0u9/.dot,d0u9,.dot,.dot The bucket of dot files. Description ...,.dot,VimL
2,1808,subscription-manager,9548.0,candlepin,candlepin/subscription-manager,candlepin,subscription-manager,subscription manager The ...,subscription-manager,Python
3,28922584,utxx,10557672.0,greenalpha,greenalpha/utxx,greenalpha,utxx,utxx an open source collection of C++ util...,utxx,C++
4,9884164,linux,2234193.0,endlessm,endlessm/linux,endlessm,linux,NaN,linux,C
5,55995561,core-project,31007006.0,khssb,khssb/core-project,khssb,core-project,NaN,core-project,Python
6,47480079,xana,11662792.0,aperaid,aperaid/xana,aperaid,xana,Laravel PHP Framework [![Build Status](https...,xana,HTML
7,56305337,ApplicationRelease,9403824.0,basti-shi031,basti-shi031/ApplicationRelease,basti-shi031,ApplicationRelease,ReadMe * python . * win How to use * ...,ApplicationRelease,Python
8,56421870,device-androidia-mixins,7951407.0,harishkrupo,harishkrupo/device-androidia-mixins,harishkrupo,device-androidia-mixins,NaN,device-androidia-mixins,Makefile
9,59026907,gitfiti,138931.0,agabardo,agabardo/gitfiti,agabardo,gitfiti,NaN,gitfiti,Python


In [110]:
readmes['repo_content'] = readmes['repo_content'].fillna(readmes['language'])
readmes['repo_content'] = readmes['repo_content'].fillna('empty')

In [111]:
readmes.head(50)

,login,name,repo_content,name_repo,language,name_repo,language_repo
0,Electronics-Ninja,BBB_python,BBB python Testing,BBB_python,Python,BBB_python,Python
1,d0u9,.dot,.dot The bucket of dot files. Description ...,.dot,VimL,.dot,VimL
2,candlepin,subscription-manager,subscription manager The ...,subscription-manager,Python,subscription-manager,Python
3,greenalpha,utxx,utxx an open source collection of C++ util...,utxx,C++,utxx,C++
4,endlessm,linux,C,linux,C,linux,C
5,khssb,core-project,Python,core-project,Python,core-project,Python
6,aperaid,xana,HTML,xana,HTML,xana,HTML
7,basti-shi031,ApplicationRelease,ReadMe python . win How to use ...,ApplicationRelease,Python,ApplicationRelease,Python
8,harishkrupo,device-androidia-mixins,Makefile,device-androidia-mixins,Makefile,device-androidia-mixins,Makefile
9,agabardo,gitfiti,Python,gitfiti,Python,gitfiti,Python


In [150]:
#Drop any projects that don't contain a README.md file 
# (Assuming these won't be good recommendations)
readmes = readmes.dropna(axis=0, subset=['readme_content'])
readmes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148608 entries, 0 to 227480
Data columns (total 10 columns):
id_repo           148608 non-null int64
name              148608 non-null object
id_user           148608 non-null float64
login_joined      148608 non-null object
user_repo         148608 non-null object
login_readme      148608 non-null object
repo              148608 non-null object
readme_content    148608 non-null object
name_repo         148605 non-null object
language          148608 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 12.5+ MB


## Try some document similarity on README.md repo files

## Create a TF-IDF Matrix for the README file content

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [5]:
readmes = pd.read_pickle('data/readme_df.pkl')
readmes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148608 entries, 0 to 227480
Data columns (total 10 columns):
id_repo           148608 non-null int64
name              148608 non-null object
id_user           148608 non-null float64
login_joined      148608 non-null object
user_repo         148608 non-null object
login_readme      148608 non-null object
repo              148608 non-null object
readme_content    148608 non-null object
name_repo         148605 non-null object
language          148608 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 12.5+ MB


In [10]:
tfidf = TfidfVectorizer(max_df=0.75)
tfidf_matrix = tfidf.fit_transform(readmes['readme_content'])

In [12]:
#print sorted(tfidf.vocabulary_)
#print tfidf_matrix.todense()
tfidf_matrix.shape

(148608, 532338)

## Find most similar repos

In [11]:
from sklearn.metrics.pairwise import linear_kernel

def find_similar(tfidf_matrix, index, top_n = 5):
    cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

In [13]:
find_similar(tfidf_matrix, 0) # BBB_python repo

[(72235, 0.41909860885853256),
 (85823, 0.38881492759066932),
 (43104, 0.38881492759066932),
 (52834, 0.37797272303810098),
 (85845, 0.33855259703736978)]

Unfortunately, the README for BBB_python is crap and taking the similarity of the tf-idf score is picking up "testing" and returning any other repo containing "testing". This is an example where a README was autogenerated and never filled out.

In [14]:
readmes.iloc[0]

id_repo                               27750972
name                                BBB_python
id_user                             7.2594e+06
login_joined                 Electronics-Ninja
user_repo         Electronics-Ninja/BBB_python
login_readme                 Electronics-Ninja
repo                                BBB_python
readme_content           BBB python   Testing 
name_repo                           BBB_python
language                                Python
Name: 0, dtype: object

In [15]:
print "Repo most similar to BBB_python:"
readmes.iloc[72235]

#1 Most similar
Repo most similar to BBB_python:


id_repo                                 4253425
name                                     python
id_user                                  565354
login_joined                      PaulFSherwood
user_repo                  PaulFSherwood/python
login_readme                      PaulFSherwood
repo                                     python
readme_content    python        python testing 
name_repo                       NeuralEvolution
language                                 Python
Name: 111805, dtype: object

In [16]:
print "#2 most similar repo"
readmes.loc[85823] #,'repo_content']

#2 most similar repo


id_repo                                                    33082177
name                      plistlib.python.standard-library.examples
id_user                                                 1.17537e+07
login_joined                       python-standard-library-examples
user_repo         python-standard-library-examples/plistlib.pyth...
login_readme                       python-standard-library-examples
repo                      plistlib.python.standard-library.examples
readme_content     !   README generated with readmemako.py (gith...
name_repo                                                CuraEngine
language                                                        C++
Name: 85823, dtype: object

## Repo similarity for a repo that actually has content

In [17]:
find_similar(tfidf_matrix, 17) # pics2pix repo

[(167, 0.99999999999999989),
 (24313, 0.27514565664810531),
 (9452, 0.25943750383419317),
 (137794, 0.23607944946336337),
 (137784, 0.23607944946336337)]

In [165]:
# pix2pix repo
readmes.loc[17]

id_repo                                                    52458198
name                                                        pix2pix
id_user                                                 2.13553e+06
login_joined                                     ruppeshnalwaya1993
user_repo                                ruppeshnalwaya1993/pix2pix
login_readme                                     ruppeshnalwaya1993
repo                                                        pix2pix
readme_content       pix pix [[Project]](https://phillipi.github...
name_repo                                                   pix2pix
language                                                        Lua
Name: 17, dtype: object

In [166]:
readmes.loc[167]

id_repo                                                    10130463
name                                               cms-sw.github.io
id_user                                                  1.7608e+06
login_joined                                                 cms-sw
user_repo                                   cms-sw/cms-sw.github.io
login_readme                                                 cms-sw
repo                                               cms-sw.github.io
readme_content    CMSSW work pages                  They include...
name_repo                                          cms-sw.github.io
language                                                 JavaScript
Name: 167, dtype: object

## Look into where these funky strings are
### Actually mot funky, just in other languages :thumbsup

In [118]:
readmes[readmes['repo_content'].str.contains("affiche")]

,login,name,repo_content,name_repo,language,name_repo,language_repo
6195,openfisca,openfisca-france,OpenFisca France [![Build Status](https://tr...,data-analysis,Python,data-analysis,Python
10216,ulefebvr,Quine,Dr Quine School project co...,sim-cran-transp,Groff,sim-cran-transp,Groff
10673,tolokoban,grenier,"Articles sur l'algorithmie, les Enigmes et a...",COSMO,Python,COSMO,Python
12267,Lahorde,pws,PWS Description Connect your Personal Wea...,handwritten-tf-1.0,Python,handwritten-tf-1.0,Python
15070,PhazCode,PSC,PSC Introduction Objectif Etant donn e...,autumn-earth,ActionScript,autumn-earth,ActionScript


## Overall, not going to use document similarity since only about 65% of repos I scraped actually had a README and many of these didn't contain any content anyway.

## Also, lots of domain specific stop words would need to be scrubbed, like 'git clone' which boost the cosine similarity between the documents but aren't meaningful similarity for my purpose of finding repo content similarity.  